# Player Data Exploration

This notebook explores the third-party player data from TruVolley and AVP sources.

## Data Sources
- **TruVolley**: Contains player ratings with `tv_id` as the identifier
- **AVP**: Contains player information with `playerProfileId` as the identifier


In [ ]:
import pandas as pd
import json
from pathlib import Path

# Set up paths
BASE_DIR = Path('../')
THIRDPARTY_DIR = BASE_DIR / 'thirdparty'


## Load TruVolley Data


In [ ]:
# Load TruVolley ratings data
truvolley_file = THIRDPARTY_DIR / 'truvolley-ratings.json'

with open(truvolley_file, 'r') as f:
    truvolley_data = json.load(f)

# Convert to DataFrame
df_truvolley = pd.DataFrame(truvolley_data)

print(f"Loaded {len(df_truvolley)} TruVolley players")
print(f"\nColumns: {list(df_truvolley.columns)}")
print(f"\nFirst few rows:")
df_truvolley.head()


Loaded 137027 TruVolley players

Columns: ['name', 'country_code', 'seeding_rating', 'global_rank', 'global_rank_change', 'country_rank', 'country_rank_change', 'gender', 'tv_id']

First few rows:


,name,country_code,seeding_rating,global_rank,global_rank_change,country_rank,country_rank_change,gender,tv_id
0,Christian Sorum,NOR,11.979612,1.0,0.0,1.0,0.0,1,7523
1,Anders Mol,NOR,11.979412,2.0,0.0,2.0,0.0,1,8606
2,Ahmed Tijan,QAT,11.915681,3.0,0.0,1.0,0.0,1,9109
3,Cherif Younousse,QAT,11.915642,4.0,0.0,2.0,0.0,1,9110
4,Jacob Nilsson,SWE,11.879244,5.0,0.0,1.0,0.0,1,22537


## Load AVP Data


In [ ]:
# Load AVP ratings data
avp_file = THIRDPARTY_DIR / 'avp-ratings.json'

with open(avp_file, 'r') as f:
    avp_data = json.load(f)

# Extract the players array from the JSON object
players_data = avp_data.get('players', [])

# Convert to DataFrame
df_avp = pd.DataFrame(players_data)

print(f"Loaded {len(df_avp)} AVP players")
print(f"\nColumns: {list(df_avp.columns)}")
print(f"\nFirst few rows:")
df_avp.head()


Loaded 36726 AVP players

Columns: ['playerProfileId', 'name', 'emails', 'ageDivision', 'gradYear', 'cityState', 'tags', 'isMale', 'points', 'events', 'profilePic', 'psrIds']

First few rows:


,playerProfileId,name,emails,ageDivision,gradYear,cityState,tags,isMale,points,events,profilePic,psrIds
0,7465,Kristen Nuss,kcnuss3@gmail.com,Adult,NaN,"New Orleans, LA","[Adult, LA]",False,4211.0,3,None,[]
1,46448,Taryn Brasher,tarynamberkloth@yahoo.com,Adult,NaN,"Sioux Falls, SD","[Adult, SD]",False,4211.0,3,None,[]
2,423,Megan Kraft,megkraft.volleyball@gmail.com,Adult,NaN,"San Diego, CA","[Adult, CA]",False,4001.0,3,None,[]
3,140691,Terese Cannon,tmcannon14@gmail.com,Adult,NaN,"Pittsford, NY","[Adult, NY]",False,4001.0,3,None,[]
4,3099,Molly Shaw,mollyturner233@gmail.com,Adult,NaN,"redondo beach, ca","[Adult, ca]",False,3807.5,3,None,[]


## Data Summary


In [ ]:
# Display basic info about both dataframes
print("=" * 50)
print("TRUVOLLEY DATA INFO")
print("=" * 50)
print(df_truvolley.info())
print(f"\nShape: {df_truvolley.shape}")
print(f"\nUnique tv_id count: {df_truvolley['tv_id'].nunique()}")

print("\n" + "=" * 50)
print("AVP DATA INFO")
print("=" * 50)
print(df_avp.info())
print(f"\nShape: {df_avp.shape}")
print(f"\nUnique playerProfileId count: {df_avp['playerProfileId'].nunique()}")


TRUVOLLEY DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 137027 entries, 0 to 137026
Data columns (total 9 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   name                 137027 non-null  object 
 1   country_code         137017 non-null  object 
 2   seeding_rating       67870 non-null   float64
 3   global_rank          67856 non-null   float64
 4   global_rank_change   67856 non-null   float64
 5   country_rank         67851 non-null   float64
 6   country_rank_change  67851 non-null   float64
 7   gender               137027 non-null  int64  
 8   tv_id                137027 non-null  int64  
dtypes: float64(5), int64(2), object(2)
memory usage: 9.4+ MB
None

Shape: (137027, 9)

Unique tv_id count: 137027

AVP DATA INFO
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 36726 entries, 0 to 36725
Data columns (total 12 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --

## Join AVP with TruVolley on Name


In [ ]:
# Create new DataFrame starting with AVP ratings
# Join TruVolley columns on exact name match
df_joined = df_avp.merge(
    df_truvolley,
    on='name',
    how='left',
    suffixes=('_avp', '_truvolley')
)

print(f"AVP DataFrame shape: {df_avp.shape}")
print(f"Joined DataFrame shape: {df_joined.shape}")
print(f"\nNumber of rows with TruVolley match: {df_joined['tv_id'].notna().sum()}")
print(f"Number of rows without TruVolley match: {df_joined['tv_id'].isna().sum()}")


AVP DataFrame shape: (36726, 12)
Joined DataFrame shape: (39958, 20)

Number of rows with TruVolley match: 34577
Number of rows without TruVolley match: 5381


In [ ]:
# Count how many TruVolley rows have exact name matches
# This means we need to count unique names that appear in both DataFrames
truvolley_names = set(df_truvolley['name'].unique())
avp_names = set(df_avp['name'].unique())
matching_names = truvolley_names.intersection(avp_names)

print(f"Unique names in TruVolley: {len(truvolley_names)}")
print(f"Unique names in AVP: {len(avp_names)}")
print(f"Exact name matches: {len(matching_names)}")
print(f"\nTruVolley rows with exact name match in AVP: {df_truvolley[df_truvolley['name'].isin(matching_names)].shape[0]}")


Unique names in TruVolley: 131041
Unique names in AVP: 36173
Exact name matches: 30808

TruVolley rows with exact name match in AVP: 33337


In [ ]:
# Display the joined DataFrame columns and a sample of matched rows
print("Joined DataFrame columns:")
print(list(df_joined.columns))
print(f"\nSample of successfully joined rows:")
df_joined[df_joined['tv_id'].notna()].head()


Joined DataFrame columns:
['playerProfileId', 'name', 'emails', 'ageDivision', 'gradYear', 'cityState', 'tags', 'isMale', 'points', 'events', 'profilePic', 'psrIds', 'country_code', 'seeding_rating', 'global_rank', 'global_rank_change', 'country_rank', 'country_rank_change', 'gender', 'tv_id']

Sample of successfully joined rows:


,playerProfileId,name,emails,ageDivision,gradYear,cityState,tags,isMale,points,events,profilePic,psrIds,country_code,seeding_rating,global_rank,global_rank_change,country_rank,country_rank_change,gender,tv_id
0,7465,Kristen Nuss,kcnuss3@gmail.com,Adult,NaN,"New Orleans, LA","[Adult, LA]",False,4211.0,3,None,[],USA,11.460835,6.0,0.0,2.0,0.0,2.0,11004.0
1,46448,Taryn Brasher,tarynamberkloth@yahoo.com,Adult,NaN,"Sioux Falls, SD","[Adult, SD]",False,4211.0,3,None,[],USA,11.460862,5.0,0.0,1.0,0.0,2.0,14794.0
2,423,Megan Kraft,megkraft.volleyball@gmail.com,Adult,NaN,"San Diego, CA","[Adult, CA]",False,4001.0,3,None,[],USA,11.115483,15.0,0.0,7.0,0.0,2.0,13877.0
3,140691,Terese Cannon,tmcannon14@gmail.com,Adult,NaN,"Pittsford, NY","[Adult, NY]",False,4001.0,3,None,[],USA,11.115889,14.0,0.0,6.0,0.0,2.0,10407.0
4,3099,Molly Shaw,mollyturner233@gmail.com,Adult,NaN,"redondo beach, ca","[Adult, ca]",False,3807.5,3,None,[],USA,11.135706,13.0,0.0,5.0,0.0,2.0,10258.0


In [ ]:
# Show some examples of exact name matches
print("Sample of exact name matches:")
sample_matches = df_joined[df_joined['tv_id'].notna()][['name', 'playerProfileId', 'tv_id', 'points', 'seeding_rating']].head(10)
sample_matches


Sample of exact name matches:


,name,playerProfileId,tv_id,points,seeding_rating
0,Kristen Nuss,7465,11004.0,4211.0,11.460835
1,Taryn Brasher,46448,14794.0,4211.0,11.460862
2,Megan Kraft,423,13877.0,4001.0,11.115483
3,Terese Cannon,140691,10407.0,4001.0,11.115889
4,Molly Shaw,3099,10258.0,3807.5,11.135706
5,Kelly Cheng,13802,8396.0,3807.5,11.149018
6,Miles Evans,4413,7603.0,3668.5,11.221864
7,Chase Budinger,1024,7587.0,3668.5,11.225337
8,Andy Benesh,1257,13085.0,3607.0,11.363877
9,Miles Partain,1644,11673.0,3607.0,11.372412


## Investigate Duplicate Names


In [ ]:
# Check for duplicate names in each DataFrame
print("TRUVOLLEY DUPLICATE NAMES:")
truvolley_name_counts = df_truvolley['name'].value_counts()
truvolley_duplicates = truvolley_name_counts[truvolley_name_counts > 1]
print(f"Total rows in TruVolley: {len(df_truvolley)}")
print(f"Unique names in TruVolley: {df_truvolley['name'].nunique()}")
print(f"Names that appear more than once: {len(truvolley_duplicates)}")
print(f"Total duplicate rows: {len(df_truvolley) - df_truvolley['name'].nunique()}")
print(f"\nTop 10 most common names in TruVolley:")
print(truvolley_duplicates.head(10))


TRUVOLLEY DUPLICATE NAMES:
Total rows in TruVolley: 137027
Unique names in TruVolley: 131041
Names that appear more than once: 4538
Total duplicate rows: 5986

Top 10 most common names in TruVolley:
name
Deleted Player      635
Unknown              11
Hannah Smith         10
Stephen Sadowski      8
Ava Johnson           8
Jordan Smith          7
Emma Miller           7
                      7
Tyler Nguyen          7
Avery Smith           6
Name: count, dtype: int64


In [ ]:
# Check for duplicate names in AVP
print("AVP DUPLICATE NAMES:")
avp_name_counts = df_avp['name'].value_counts()
avp_duplicates = avp_name_counts[avp_name_counts > 1]
print(f"Total rows in AVP: {len(df_avp)}")
print(f"Unique names in AVP: {df_avp['name'].nunique()}")
print(f"Names that appear more than once: {len(avp_duplicates)}")
print(f"Total duplicate rows: {len(df_avp) - df_avp['name'].nunique()}")
print(f"\nTop 10 most common names in AVP:")
print(avp_duplicates.head(10))


AVP DUPLICATE NAMES:
Total rows in AVP: 36726
Unique names in AVP: 36173
Names that appear more than once: 507
Total duplicate rows: 553

Top 10 most common names in AVP:
name
Isabella Martinez    6
Addison Williams     4
Justin Lee           4
Ryan Smith           4
Epic Fierce          4
Isabella Johnson     3
Olivia Taylor        3
Megan Rice           3
Madison Davis        3
Hannah Smith         3
Name: count, dtype: int64


In [ ]:
# Check why the join created more rows
# The issue is that when a name appears multiple times in TruVolley,
# each AVP row with that name gets matched to ALL TruVolley rows with that name
print(f"AVP rows: {len(df_avp)}")
print(f"Joined rows: {len(df_joined)}")
print(f"Difference: {len(df_joined) - len(df_avp)}")
print(f"\nThis means the join is creating {len(df_joined) - len(df_avp)} extra rows")
print(f"due to duplicate names in TruVolley.")

# Example: Show a specific name that appears multiple times in TruVolley
example_duplicate_name = truvolley_duplicates.index[0] if len(truvolley_duplicates) > 0 else None
if example_duplicate_name:
    print(f"\nExample: '{example_duplicate_name}' appears {truvolley_duplicates[example_duplicate_name]} times in TruVolley")
    print(f"\nTruVolley rows for '{example_duplicate_name}':")
    print(df_truvolley[df_truvolley['name'] == example_duplicate_name][['name', 'tv_id', 'gender', 'country_code']])
    if example_duplicate_name in df_avp['name'].values:
        print(f"\nAVP rows for '{example_duplicate_name}':")
        print(df_avp[df_avp['name'] == example_duplicate_name][['name', 'playerProfileId', 'isMale']])
        print(f"\nIn the joined DataFrame, each AVP row creates {truvolley_duplicates[example_duplicate_name]} rows")


AVP rows: 36726
Joined rows: 39958
Difference: 3232

This means the join is creating 3232 extra rows
due to duplicate names in TruVolley.

Example: 'Deleted Player' appears 635 times in TruVolley

TruVolley rows for 'Deleted Player':
                  name   tv_id  gender country_code
14278   Deleted Player  103895       2          USA
34812   Deleted Player  110775       0          USA
45600   Deleted Player   64355       2          USA
51520   Deleted Player  111013       0          USA
53094   Deleted Player  110774       0          USA
...                ...     ...     ...          ...
126429  Deleted Player   18658       2          USA
126432  Deleted Player   18661       2          USA
126433  Deleted Player   18662       2          USA
126631  Deleted Player   18401       2          USA
135344  Deleted Player   10766       2          USA

[635 rows x 4 columns]


## Create Deduplicated DataFrames with Highest Ranked Players


In [ ]:
# Deduplicate AVP by keeping the player with highest points per name
# For ties, we'll keep the first occurrence
df_avp_unique = df_avp.sort_values('points', ascending=False).drop_duplicates(subset='name', keep='first')

print(f"Original AVP rows: {len(df_avp)}")
print(f"Unique names in AVP: {df_avp['name'].nunique()}")
print(f"Deduplicated AVP rows: {len(df_avp_unique)}")
print(f"\nSample of deduplicated AVP (top by points):")
df_avp_unique[['name', 'playerProfileId', 'points', 'isMale']].head()


Original AVP rows: 36726
Unique names in AVP: 36173
Deduplicated AVP rows: 36173

Sample of deduplicated AVP (top by points):


,name,playerProfileId,points,isMale
0,Kristen Nuss,7465,4211.0,False
1,Taryn Brasher,46448,4211.0,False
2,Megan Kraft,423,4001.0,False
3,Terese Cannon,140691,4001.0,False
4,Molly Shaw,3099,3807.5,False


In [ ]:
# Deduplicate TruVolley by keeping the player with best (lowest) global_rank per name
# Lower rank number = better (rank 1 is best)
# For names without rank, we'll sort by seeding_rating (higher is better) as secondary
# Then drop duplicates, keeping the first (best ranked)
df_truvolley_unique = df_truvolley.copy()
# Create a sort key: use global_rank if available, otherwise use negative seeding_rating for sorting
# (NaN values will sort last, so players with ranks come first)
df_truvolley_unique['_sort_rank'] = df_truvolley_unique['global_rank'].fillna(999999)
df_truvolley_unique = df_truvolley_unique.sort_values(['_sort_rank', 'seeding_rating'], ascending=[True, False])
df_truvolley_unique = df_truvolley_unique.drop_duplicates(subset='name', keep='first')
df_truvolley_unique = df_truvolley_unique.drop(columns=['_sort_rank'])

print(f"Original TruVolley rows: {len(df_truvolley)}")
print(f"Unique names in TruVolley: {df_truvolley['name'].nunique()}")
print(f"Deduplicated TruVolley rows: {len(df_truvolley_unique)}")
print(f"\nSample of deduplicated TruVolley (top ranked):")
df_truvolley_unique[['name', 'tv_id', 'global_rank', 'seeding_rating', 'gender']].head()


Original TruVolley rows: 137027
Unique names in TruVolley: 131041
Deduplicated TruVolley rows: 131041

Sample of deduplicated TruVolley (top ranked):


,name,tv_id,global_rank,seeding_rating,gender
0,Christian Sorum,7523,1.0,11.979612,1
13,Arthur Lanci,9209,1.0,11.642075,2
1,Anders Mol,8606,2.0,11.979412,1
17,Nina Brunner,7313,2.0,11.507112,2
2,Ahmed Tijan,9109,3.0,11.915681,1


## Join Deduplicated DataFrames


In [ ]:
# Rename columns with prefixes before joining
# AVP columns get 'avp_' prefix (except 'name')
df_avp_renamed = df_avp_unique.copy()
df_avp_renamed.columns = ['avp_' + col if col != 'name' else col for col in df_avp_renamed.columns]

# TruVolley columns get 'tv_' prefix (except 'name')
df_truvolley_renamed = df_truvolley_unique.copy()
df_truvolley_renamed.columns = ['tv_' + col if col != 'name' else col for col in df_truvolley_renamed.columns]

# Join the deduplicated DataFrames on name
df_final = df_avp_renamed.merge(
    df_truvolley_renamed,
    on='name',
    how='left'
)

print(f"AVP unique rows: {len(df_avp_unique)}")
print(f"TruVolley unique rows: {len(df_truvolley_unique)}")
print(f"Final joined DataFrame rows: {len(df_final)}")
print(f"\nNumber of rows with TruVolley match: {df_final['tv_tv_id'].notna().sum()}")
print(f"Number of rows without TruVolley match: {df_final['tv_tv_id'].isna().sum()}")
print(f"\nFinal DataFrame columns: {list(df_final.columns)}")


AVP unique rows: 36173
TruVolley unique rows: 131041
Final joined DataFrame rows: 36173

Number of rows with TruVolley match: 30808
Number of rows without TruVolley match: 5365

Final DataFrame columns: ['playerProfileId', 'name', 'emails', 'ageDivision', 'gradYear', 'cityState', 'tags', 'isMale', 'points', 'events', 'profilePic', 'psrIds', 'country_code', 'seeding_rating', 'global_rank', 'global_rank_change', 'country_rank', 'country_rank_change', 'gender', 'tv_id']


In [ ]:
# Display sample of the final joined dataset
print("Sample of final joined dataset (with TruVolley match):")
df_final[df_final['tv_tv_id'].notna()][
    ['name', 'avp_playerProfileId', 'tv_tv_id', 'avp_points', 'tv_global_rank', 'tv_seeding_rating', 'avp_isMale', 'tv_gender']
].head(10)


In [ ]:
# Summary statistics
print("=" * 50)
print("FINAL DATASET SUMMARY")
print("=" * 50)
print(f"Total players in final dataset: {len(df_final)}")
print(f"Players with both AVP and TruVolley data: {df_final['tv_tv_id'].notna().sum()}")
print(f"Players with only AVP data: {df_final['tv_tv_id'].isna().sum()}")
print(f"\nMatch rate: {df_final['tv_tv_id'].notna().sum() / len(df_final) * 100:.2f}%")
print(f"\nUnique names in final dataset: {df_final['name'].nunique()}")
print(f"Any duplicate names? {len(df_final) != df_final['name'].nunique()}")
